<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-09-06 20:57:03


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['MCDOWELL-N','IBREALEST','CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS', 'UJJIVAN']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin

In [5]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='max')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_v20 = df_v20.append(df_tmp)
        df_v20 = pd.concat([df_v20, df_tmp])

    return df_v20.sort_index()

In [6]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [7]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [8]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'AV40N') | (df_v20_prospects['Criteria'] == 'AV40')]

In [9]:
df_final_v20 = pd.concat([df_v20_prospects_V40N.reset_index(), df_v20_prospects_200SMA.reset_index()])
df_final_v20.set_index("Date", inplace = True)
df_final_v20.sort_index()

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2006-06-09 00:00:00+05:30,BAJAJHIND.NS,43.27,266.0,258.118103,XR
2006-08-02 00:00:00+05:30,BAJAJHIND.NS,20.80,289.0,270.067627,XR
2007-02-02 00:00:00+05:30,NETWORK18.NS,27.78,657.0,364.143036,NaN
2007-02-07 00:00:00+05:30,NETWORK18.NS,24.26,659.0,423.114166,NaN
2007-02-13 00:00:00+05:30,TV18BRDCST.NS,23.07,104.0,87.111298,XR
...,...,...,...,...,...
2022-07-22 00:00:00+05:30,HAPPSTMNDS.NS,22.80,1086.0,983.168579,AR
2023-05-23 00:00:00+05:30,AWL.NS,33.52,566.0,488.700012,XSP
2023-05-24 00:00:00+05:30,VALIANTORG.NS,20.22,573.0,552.450012,XR


In [10]:
df_final_v20.sort_index().tail(20)

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2020-06-23 00:00:00+05:30,BANDHANBNK.NS,37.52,400.0,344.728241,XSP
2020-06-25 00:00:00+05:30,SAMMAANCAP.NS,21.14,198.0,193.263702,XR
2020-07-07 00:00:00+05:30,BANDHANBNK.NS,27.62,395.0,387.813110,XSP
2020-08-20 00:00:00+05:30,ZEEL.NS,27.07,205.0,194.201340,NaN
2021-04-09 00:00:00+05:30,VALIANTORG.NS,21.91,1474.0,1467.968262,XR
2021-05-24 00:00:00+05:30,ASIANTILES.NS,20.30,152.0,121.733452,XR
2021-09-09 00:00:00+05:30,ASIANTILES.NS,25.42,130.0,118.388573,XR
2022-01-10 00:00:00+05:30,VALIANTORG.NS,23.21,1366.0,1277.878418,XR
2022-01-27 00:00:00+05:30,LUXIND.NS,25.10,3472.0,2787.197266,X40N


In [11]:
df_v20_prospects['Criteria'].value_counts()

,count
Criteria,
XR,182
XSP,58
AR,46
X40N,17
SR,4
MGC,4
EXP50,1
